In [1]:
import urllib2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from urlparse import urlsplit, urljoin
from bs4 import BeautifulSoup

In [6]:
base_url = 'http://www.cra-arc.gc.ca/formspubs/t1gnrl/llyrs-eng.html'
base_page = urllib2.urlopen(base_url).read()
selection = BeautifulSoup(base_page, 'html.parser')
# print selection.prettify()

<!DOCTYPE html>
<!--[if lt IE 9]><html class="no-js lt-ie9" lang="en"><![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <title>
   Tax packages for all years
  </title>
  <meta content="Tax packages for all years" name="dcterms.title"/>
  <meta content="General income tax and benefits packages from 1985 to 2013. Each package includes the guide, the return, and related schedules, and the provincial information and forms." name="description"/>
  <meta content="General income tax and benefits packages from 1985 to 2013. Each package includes the guide, the return, and related schedules, and the provincial information and forms." name="dcterms.description"/>
  <meta content="Tax Package, Tax filing, Income tax, Guide, Return, Forms, Schedules, T1, Benefits" name="keywords"/>
  <meta content="Government of Canada, Canada Revenue Agency" name="dcterms.creator"/>
  <meta content="Government of Canada, Canada Revenue Agency

### Get a url for each year

In [43]:
year_items = selection.find(property="mainContentOfPage").ul.find_all('li')
year_pages_dict = {}
year = 2014
for item in year_items:
    year_pages_dict[str(year)] = item.a.get("href")
    year = year - 1
year_pages_dict

{'1985': u'/formspubs/prioryear/t1/1985/menu-e.html',
 '1986': u'/formspubs/prioryear/t1/1986/menu-e.html',
 '1987': u'/formspubs/prioryear/t1/1987/menu-e.html',
 '1988': u'/formspubs/prioryear/t1/1988/menu-e.html',
 '1989': u'/formspubs/prioryear/t1/1989/menu-e.html',
 '1990': u'/formspubs/prioryear/t1/1990/menu-e.html',
 '1991': u'/formspubs/prioryear/t1/1991/menu-e.html',
 '1992': u'/formspubs/prioryear/t1/1992/menu-e.html',
 '1993': u'/formspubs/prioryear/t1/1993/menu-e.html',
 '1994': u'/formspubs/prioryear/t1/1994/menu-e.html',
 '1995': u'/formspubs/prioryear/t1/1995/menu-e.html',
 '1996': u'/formspubs/prioryear/t1/1996/menu-e.html',
 '1997': u'/formspubs/prioryear/t1/1997/menu-e.html',
 '1998': u'/formspubs/prioryear/t1/1998/menu-e.html',
 '1999': u'/formspubs/prioryear/t1/1999/menu-e.html',
 '2000': u'/formspubs/prioryear/t1/2000/menu-e.html',
 '2001': u'/formspubs/prioryear/t1/2001/menu-e.html',
 '2002': u'/formspubs/prioryear/t1/2002/menu-e.html',
 '2003': u'/formspubs/priory

In [ ]:
url_split = urlsplit(base_url)
year_pages_dict_full = {}
for year, url in year_pages_dict.iteritems():
    # Create the url and then append it into the empty array
    url_joined = 'http://' + url_split.netloc + url
    year_pages_dict_full[year] = url_joined

# Debugging
year_pages_dict_full

In [ ]:
provincial_links = {}

def provinceUrlListAppend(url_list, flag, year_url, year):
    print year
    if flag == "normal" or flag == "older":
        for url in url_list:
            provincial_links[year].append('http://' + url_split.netloc + url)
    elif flag == "reallyOld":
        for url in url_list:
            link = 'http://' + url_split.netloc
            link += urlsplit(year_url).path.rsplit('/', 1)[0]
            link += "/" + url
            provincial_links[year].append(link)

def provinceUrlListProcess(url_list, year_url, year):
    provincial_links[year] = []
    # If it's 'normal' and Alberta is detected, go ahead
    if url_list[0] == 'Alberta' or url_list[0] == ' ':
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[1]/li/a/@href").extract()
        provinceUrlListAppend(url_list, "normal", year_url, year)
    else:
        # In some older links, they stuck another unordered list above
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[2]/li/a/@href").extract()
        # Another edge case check where they started using relative urls around 1998
        if url_list[0][0] == "/":
            provinceUrlListAppend(url_list, "older", year_url, year)
        else:
            provinceUrlListAppend(url_list, "reallyOld", year_url, year)

for year, year_url in year_pages_dict_full.iteritems():
    year_page = urllib2.urlopen(year_url).read()
    selection = Selector(text=year_page)
    provt_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/text()").extract()
    provinceUrlListProcess(provt_pages, year_url, year)

print provincial_links

In [ ]:
year = provincial_links["2014"]
url = year[0]
print url
page = urllib2.urlopen(url).read()
selection = Selector(text=base_page)
selection